In [1]:
from functools import partial
from pathlib import Path
from typing import Dict, Union

import pymem3dg as dg
import pymem3dg.boilerplate as dgb
from process_bigraph import Process, ProcessTypes

from bsp import app_registrar


# pymem3dg.boilerplate.preferredAreaSurfaceTensionModel¶
# preferredVolumeOsmoticPressureModel


class MembraneProcess(Process):
    """
    :config mesh_file: input .ply file TODO: generalize this
    :config pressure_models: list[dict[str, float | str]] {name: str, **params}, for example {preferredAreaSurfaceTensionModel, area=, modulus=, prefferedArea
    :config parameters: dict[str, float]: {**dir(pymem3dg.Parameters()}: 'adsorption', 'aggregation', 'bending', 'boundary', 'damping', 'dirichlet', 'dpd',
        entropy', 'external', 'osmotic', 'point', 'protein', 'proteinMobility', 'selfAvoidance', 'spring', 'temperature', 'tension', 'variation'.

    """
    config_schema = {
        'mesh_file': 'string',
        'tension_,
        'parameters': 'tree[float]'
    }

    def __init__(self, config: Dict[str, Union[Dict[str, float], str]] = None, core: ProcessTypes = None):
        super().__init__(config, core)

        # create geometry from model file
        self.geometry = dg.Geometry(self.config["mesh_file"])

        # create and set parameters (non-pressure model)
        self.parameters = dg.Parameters()
        param_spec = self.config.get("parameters")
        if param_spec:
            for attribute_name, attribute_spec in param_spec.items():  # ie: adsorption, aggregiation, bending, etc
                attribute = getattr(self.parameters, attribute_name)
                for name, value in attribute_spec.items():
                    setattr(attribute, name, value)

        


prolate = "../fixtures/sample_meshes/prolate.ply"

config = {
    'mesh_file': "../fixtures/sample_meshes/oblate.ply",
    'parameters': {
        'bending': {
            'Kbc': 8.22e-5
        }
    }
}

process = MembraneProcess(config=config, core=app_registrar.core)

In [2]:
process

In [ ]:
oblate = "../sample_meshes/oblate.ply"
prolate = "../sample_meshes/prolate.ply"

In [ ]:
p = dg.Parameters()

p.bending.Kbc = 8.22e-5

p.tension.form = partial(
    dgb.preferredAreaSurfaceTensionModel, modulus=0.1, preferredArea=12.4866
)
p.osmotic.form = partial(
    dgb.preferredVolumeOsmoticPressureModel,
    preferredVolume=0.7 * 3.14 * 4 / 3,
    reservoirVolume=0,
    strength=0.02,
)

In [ ]:
geo = dg.Geometry(oblate)

outputDir = Path("output/tutorial1/biconcave")
outputDir.mkdir(parents=True, exist_ok=True)

g = dg.System(geometry=geo, parameters = p)
g.initialize()

fe = dg.Euler(system = g, characteristicTimeStep = 2,
              totalTime = 10000, savePeriod = 100,
              tolerance = 1e-11, outputDirectory = str(outputDir))
# Uncomment to print intermediate statistics
fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
success = fe.integrate()


In [ ]:
# plot the netcdf data in outputDir/"traj.nc" in matplotlib
import pymem3dg.visual as dg_vis
import matplotlib.pyplot as plt

from netCDF4 import Dataset

data = Dataset(str(outputDir / "traj.nc"),'r')
# get data parameters
data.groups['Trajectory'].variables

# get the coordinates from groups/trajectory.variables
x = data.groups['Trajectory'].variables['coordinates'][:]

# plot x[] in 3d spaces
# where the array is xyz, xyz, xyz

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(x[0][::3], x[0][1::3], x[0][2::3])
ax.scatter(x[-1][::3], x[-1][1::3], x[-1][2::3])

plt.show()